# 확률 기반 (통계적)

### Apriori

In [22]:
pip install mlxtend

In [13]:
import mlxtend
import numpy as np
import pandas as pd

In [14]:
data = np.array([
    ['우유', '기저귀', '쥬스'],
    ['양상추', '기저귀', '맥주'],
    ['우유', '양상추', '기저귀', '쥬스'],
    ['양상추', '맥주']
])

<ipython-input-14-0dcb8ff8517a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array([


In [15]:
from mlxtend.preprocessing import TransactionEncoder

In [16]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,False,True,True,True
3,False,True,True,False,False


In [19]:
%%time
from mlxtend.frequent_patterns import apriori

Wall time: 12.3 ms


In [21]:
apriori(df, min_support=0.5, use_colnames=True)

,support,itemsets
0,0.75,(기저귀)
1,0.50,(맥주)
2,0.75,(양상추)
3,0.50,(우유)
4,0.50,(쥬스)
5,0.50,"(기저귀, 양상추)"
6,0.50,"(기저귀, 우유)"
7,0.50,"(쥬스, 기저귀)"
8,0.50,"(맥주, 양상추)"
9,0.50,"(쥬스, 우유)"


## 추론적 방법 (유사도 기반)

### WORD2VEC

In [ ]:
#### skip-gram (특정 단어 앞 뒤에 있는 단어 추론)

In [2]:
!pip install gensim

In [3]:
from gensim.models import Word2Vec

In [4]:
doc = [
    'You say goodbye and I say hello',
    'I say happy or he said white'
]

In [5]:
sentences = [sentence.split(' ') for sentence in doc]
## [처리내용 for one in 복수형태의자료구조] => 항상 결과는 리스트

In [6]:
sentences

[['You', 'say', 'goodbye', 'and', 'I', 'say', 'hello'],
 ['I', 'say', 'happy', 'or', 'he', 'said', 'white']]

In [7]:
model = Word2Vec(window=1, min_count=1, sg=1)
model.build_vocab(sentences)
model.wv.most_similar('say')

[('goodbye', 0.21617145836353302),
 ('hello', 0.09291722625494003),
 ('You', 0.06285080313682556),
 ('and', 0.0270574688911438),
 ('happy', 0.016134707257151604),
 ('I', -0.01083917822688818),
 ('he', -0.027750367298722267),
 ('white', -0.052346762269735336),
 ('or', -0.05987626686692238),
 ('said', -0.1116705909371376)]

In [ ]:
### 유사도 측정 방법
## -유사도- 
## 유클리디안 거리
## 코사인 거리
## 피어슨 거리

## 뭘 쓰건 값은 살짝 다르지만 결과는 비슷함

## 협업 필터링

### KNN

In [ ]:
# 유저 따로 아이템 따로 나눠서 각각 특성만 고려해서 추론 -> KNN
# 유저와 아이템 함께 고려해서 (다중회귀와 비슷하게) -> SVD

### SVD (잠재 요인 기법)

In [ ]:
# 고유값 분해 : 
#원래의 사용자/제품/평가가 들어있는 하나의 행렬을 여러개로 나누었음. 
#원래의 성질은 변하지 않음

In [23]:
!pip install scikit-surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=751503 sha256=f8c1e5e7903dd33d81225453d6e0627bbbeee0e827b1286ba80bba009e56ee2f
  Stored in directory: c:\users\82109\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise


In [25]:
import surprise

In [34]:
print(surprise.__version__)

1.1.1


In [38]:
# 고유값 분해(행렬을 더 낮은 차원으로 분해)해서 
# 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import SVD

# SVD를 사용하기 위한 테이터셋을 만들어주는 클래스
# 사용자, 아이템, 평점
from surprise import Dataset

# RMSE, MAE 등을 사용한 정확도 측정
from surprise import accuracy

# 훈련/검증 데이터 분류
from surprise.model_selection import train_test_split

In [ ]:
## 1. 데이터셋을 만들어주자(사용자, 아이템, 평점)

In [39]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [40]:
## 2. SVD() 모델 선정
algo = SVD()

In [41]:
## 3. 훈련용 데이터로 fit() (학습)
algo.fit(trainset)

In [42]:
## 4. 검증용 데이터로 예측
predictions = algo.test(testset)
print('prediction type : ', type(predictions), ' size : ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'>  size :  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.476735077156584, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.5980342669535235, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.918390089641869, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.55664850708442, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.361423393190703, details={'was_impossible': False})]

In [37]:
# 5. 정확도 계산
# uid = , iid = , est = 
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.6142518901528145),
 ('882', '291', 3.704279873054159),
 ('535', '507', 3.923910949874137)]

In [ ]:
# 6. 사용자, 아이템 정보 주고 예측 실행